# Gen Bill  

In [ ]:
import os
import time
import random
from datetime import datetime, timedelta
from faker import Faker
from dotenv import load_dotenv
import requests
from docx import Document
from docx.shared import Pt, Cm
from docx.enum.text import WD_ALIGN_PARAGRAPH
from horde_sdk.ai_horde_api.ai_horde_clients import AIHordeAPISimpleClient
from horde_sdk.ai_horde_api.apimodels import ImageGenerateAsyncRequest, ImageGenerationInputPayload
from horde_sdk.ai_horde_api import KNOWN_SAMPLERS
from PIL import Image
from io import BytesIO

# Load environment variables
load_dotenv()
ts = int(time.time())
# Initialize Faker
fake = Faker("fr_FR")

# Directory to save generated documents
OUTPUT_DIR = "generated_documents"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Retrieve Horde API Key
STABLE_HORDE_API_KEY = os.getenv("STABLE_HORDE_API_KEY")


def generate_fake_hospital_name():
    """Generate a random hospital name with a realistic format."""
    suffixes = ["General Hospital", "Medical Center", "Clinic", "Health Center", "Care Center", "Memorial Hospital"]
    name = f"{random.choice([fake.first_name(), fake.last_name()])} {random.choice(suffixes)}"
    return name, name.replace(" ", "_").lower()


def generate_image(prompt, filename):
    """Generate an image using AI Horde and save it to disk."""
    client = AIHordeAPISimpleClient()
    request = ImageGenerateAsyncRequest(
        apikey=STABLE_HORDE_API_KEY,
        prompt=prompt,
        models=["Deliberate"],
        params=ImageGenerationInputPayload(
            width=448,
            height=448,
            sampler_name=KNOWN_SAMPLERS.k_euler_a,
            clip_skip=1,
            n=1
        ),
    )
    response, _ = client.image_generate_request(request)

    if hasattr(response, "generations") and response.generations:
        image_url = response.generations[0].img
        image_path = os.path.join("images", f"{filename}.png")
        download_image(image_url, image_path)
        return image_path
    return None

def download_image(url, path):
    """Download an image from a URL and save it as PNG."""
    response = requests.get(url)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        # Convert to PNG if needed
        try:
            image = Image.open(BytesIO(response.content))
            png_path = path.replace(".webp", ".png")
            image.convert("RGB").save(png_path, "PNG")
            print(f"✅ Image saved to {png_path}")
            return png_path
        except Exception as e:
            print(f"❌ Failed to process image: {e}")
            return None
    else:
        print(f"❌ Failed to download image from {url}")
        return None

def generate_assurance_maladie_documents():
    """
    Generates a DOCX file resembling the Assurance Maladie statement
    with all PII (names, addresses, amounts, dates) randomly generated by Faker.
    """
    ASSURANCE_MALADIE_LOGO = logo_path
    doc = Document()

    # Set up default font
    style = doc.styles["Normal"]
    style.font.name = "Calibri"
    style.font.size = Pt(11)

    # ---- 1) HEADER SECTION ----
    # Page 1/1 in top-right corner
    paragraph = doc.add_paragraph()
    paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    paragraph.add_run("Page 1/1")

    # Optionally add the Assurance Maladie logo
    if os.path.exists(ASSURANCE_MALADIE_LOGO):
        doc.add_picture(ASSURANCE_MALADIE_LOGO, width=Cm(4))
    else:
        # Fallback heading
        heading = doc.add_paragraph()
        heading_run = heading.add_run("l’Assurance Maladie")
        heading_run.bold = True
        heading_run.font.size = Pt(14)

    # A small note or slogan
    doc.add_paragraph("Agir ensemble, protéger chacun")

    # Insured info
    insured_name = fake.name()
    # Generate a random SSN-like number (fake FR format)
    numero_secu = fake.bothify(text="1 ## ## ?? ## ### ??? ##", letters="AB")

    doc.add_paragraph(
        f"assuré social {insured_name.upper()}\n"
        f"n° de Sécurité Sociale {numero_secu}\n"
        "Pour toutes vos démarches, utilisez le compte ameli\n"
        "ou l’application ameli pour smartphone"
    )

    # Caisse d'assurance maladie address block (static example text, can remain unchanged or randomize)
    doc.add_paragraph(
        "Caisse d'assurance maladie de Loire-Atlantique   9, rue Gaston\n"
        "Rondeau\n"
        "44958 NANTES CEDEX 9"
    )

    # ---- 2) MAIN BODY ----
    # Address block (recipient) - random address
    address_line1 = insured_name.upper()
    address_line2 = fake.street_address().upper()
    address_line3 = f"{fake.postcode()} {fake.city().upper()}"

    doc.add_paragraph(f"{address_line1}\n{address_line2}\n{address_line3}")

    # “Voici le détail...” paragraph with random statement date
    statement_date = (datetime.now() + timedelta(days=random.randint(1, 10))).strftime("%d/%m/%Y")
    doc.add_paragraph(
        f"Voici le détail des versements vous concernant pour la journée du {statement_date}\n"
        "Ces informations n'ont pas été transmises à un organisme complémentaire. "
        "Si vous en avez un, pensez à lui envoyer ce relevé et conserver une copie."
    )

    # ---- 3) TABLE OF PRESTATIONS ----
    table = doc.add_table(rows=1, cols=6)
    table.style = "Table Grid"

    # Table Header
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = "dates"
    hdr_cells[1].text = "nature des prestations"
    hdr_cells[2].text = "montant payé"
    hdr_cells[3].text = "rembours."
    hdr_cells[4].text = "taux"
    hdr_cells[5].text = "montant versé"

    # Example possible medical procedures (you can customize the list)
    medical_procedures = [
        "RACHIS NON OPERE (RAM 7.4B)\nfranchise à retenir (FR1)",
        "CONSULTATION MÉDECIN GÉNÉRALISTE\nfranchise à retenir (FR2)",
        "VACCINATION ANTIGRIPPALE\nfranchise à retenir (FR1)",
        "RADIOGRAPHIE THORACIQUE\nfranchise à retenir (FR1)"
    ]

    # Let's generate between 2 and 4 random rows
    num_rows = random.randint(2, 4)
    for _ in range(num_rows):
        row_cells = table.add_row().cells
        # Random date (within past few months)
        service_date = (datetime.now() - timedelta(days=random.randint(1, 60))).strftime("%d/%m/%Y")
        
        # Random choice of medical procedure
        nature = random.choice(medical_procedures)

        # Random amounts
        montant_paye = round(random.uniform(10, 100), 2)
        base_rembourse = montant_paye  # In a real scenario, might differ
        taux_remboursement = random.choice([50, 60, 70, 80, 90])
        montant_verse = round(base_rembourse * (taux_remboursement / 100.0), 2)

        row_cells[0].text = service_date
        row_cells[1].text = nature
        # Format floats as strings with comma or dot
        row_cells[2].text = f"{montant_paye:.2f}".replace(".", ",")
        row_cells[3].text = f"{base_rembourse:.2f}".replace(".", ",")
        row_cells[4].text = f"{taux_remboursement} %"
        row_cells[5].text = f"{montant_verse:.2f}".replace(".", ",")

    doc.add_paragraph()

    # Payment detail
    pay_date = (datetime.now() + timedelta(days=random.randint(1, 10))).strftime("%d/%m/%Y")
    destinataire_name = fake.name().upper()
    total_paid = round(random.uniform(15, 200), 2)
    payment_paragraph = (
        f"réglé le {pay_date} au destinataire {destinataire_name} : "
        f"{total_paid:.2f} euro(s)"
    )
    doc.add_paragraph(payment_paragraph)

    # ---- 4) FOOTNOTE OR DISCLAIMER ----
    disclaimer = doc.add_paragraph()
    disclaimer_run = disclaimer.add_run(
        "(1) Les participations forfaitaires et franchises constatées sur ces actes "
        "sont déduites du montant remboursé dans la partie « Vos remboursements de soins » "
        "ou le seront sur vos prochains règlements."
    )
    disclaimer_run.font.size = Pt(8)

    # Unique timestamp for filenames
    ts = int(time.time())

    # Save DOCX
    docx_name = f"assurance_maladie_statement_{ts}.docx"
    docx_path = os.path.join(OUTPUT_DIR, docx_name)
    doc.save(docx_path)
    print(f"DOCX generated: {docx_path}")

    # Convert to PDF if the .docx exists
    pdf_name = f"assurance_maladie_statement_{ts}.pdf"
    pdf_path = os.path.join(OUTPUT_DIR, pdf_name)

    if os.path.exists(docx_path):
        convert(docx_path, pdf_path)
        print(f"Converted {docx_path} to {pdf_path}")
    else:
        print(f"Error: {docx_path} does not exist. Cannot convert to PDF.")


def generate_medical_bill():
    """Generate a medical bill document with fake PII and AI-generated images."""
    # Generate dynamic data
    # signature_path = generate_image("A simple hand-written signature on a white background", f"doctor_signature_{ts}")

    # Create DOCX document
    doc = Document()
    style = doc.styles["Normal"]
    style.font.name = "Calibri"
    style.font.size = Pt(11)

    # Add logo
    if logo_path and os.path.exists(logo_path):
        doc.add_picture(logo_path, width=Pt(100))
    else:
        doc.add_paragraph(hospital_name).runs[0].bold = True

    # Add title and header information
    doc.add_paragraph(f"RADIOLOGIE ET IMAGERIE MÉDICALE\n{hospital_name}").alignment = WD_ALIGN_PARAGRAPH.LEFT

    doc.add_paragraph(
        f"Chef de pôle : Pr {fake.name()}\n"
        f"Chef de service : Dr {fake.name()} - Adjointe : Dr {fake.name()}"
    )

    # Add list of doctors
    doctors = ", ".join(f"Dr {fake.name()}" for _ in range(6))
    doc.add_paragraph(doctors)

    # Add current date
    doc.add_paragraph(f"Nantes, le {datetime.now().strftime('%d/%m/%Y')}")

    # Add barcode
    doc.add_paragraph(fake.ean13())

    # Add patient information
    doc.add_paragraph(f"{fake.name()}\n{fake.address().replace('\n', ', ')}").runs[0].bold = True

    # Add prescribing doctor and appointment details
    doc.add_paragraph(f"À la demande du Docteur {fake.name()},")
    future_date = datetime.now() + timedelta(days=random.randint(5, 30))
    doc.add_paragraph(f"Veuillez vous présenter le : {future_date.strftime('%A %d %B %Y à %H:%M')}").alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Add instructions
    doc.add_paragraph(
        f"Depuis l'accès principal du {hospital_name} - site Hôtel Dieu (Place Alexis).\n"
        "Suivre les indications pour Service Imagerie Médicale et se présenter à l'accueil A."
    )

    # Add administrative document info
    doc.add_paragraph(
        "Veuillez déposer vos documents administratifs sur l'application Digihosp PATIENT ou le site digihosp.fr."
    )

    # Add new policy info
    doc.add_paragraph(
        "À partir du 1er mars 2024, les produits de contraste pour scanner et IRM seront directement fournis par l'hôpital."
    )

    # Add cancellation info
    email = fake.email()
    phone = fake.phone_number()
    doc.add_paragraph(
        f"Pour toute annulation, contactez-nous par email: {email} ou par téléphone: {phone}."
    )

    # Add footer
    doc.add_paragraph(
        f"{hospital_name} - Hôtel-Dieu\n"
        "1, place Alexis Ricordeau, 44093 Nantes Cedex 1\n"
        "Secrétariat: 02 40 08 41 70 - Fax: 02 40 08 41 92\n"
        f"www.{sanitized_hospital_name}.fr"
    ).alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Add doctor's signature
    if signature_path and os.path.exists(signature_path):
        doc.add_paragraph("Signature du docteur:")
        doc.add_picture(signature_path, width=Pt(100))
    else:
        doc.add_paragraph("[Signature non disponible]")

    # Save document
    timestamp = int(time.time())
    output_path = os.path.join(OUTPUT_DIR, f"medical_bill_{sanitized_hospital_name}_{timestamp}.docx")
    doc.save(output_path)
    print(f"📄 Document saved to: {output_path}")

def generate_document(type= 'insurance'):
  hospital_name, sanitized_hospital_name = generate_fake_hospital_name()
  logo_path = generate_image(f"A professional medical hospital logo for {hospital_name}", f"hospital_logo_{ts}")
  signature_path = './images/doctor_signature.png'
  
  match type:
    case 'insurance':
      return generate_assurance_maladie_documents()
    case 'medical_bill':
      return generate_medical_bill()
    case _:
      return generate_assurance_maladie_documents()

In [32]:
generate_document()

2025-02-17 20:44:03.200 | PROGRESS | horde_sdk.ai_horde_api.ai_horde_clients:image_generate_request:793 - Requesting 1 images.
2025-02-17 20:44:03.202 | DEBUG    | horde_sdk.ai_horde_api.ai_horde_clients:_do_request_with_check:660 - Submitting request: {'trusted_workers': False, 'slow_workers': True, 'extra_slow_workers': False, 'workers': [], 'worker_blacklist': [], 'models': ['Deliberate'], 'dry_run': False, 'accept': <GenericAcceptTypes.json: 'application/json'>, 'client_agent': 'horde_sdk:v0.17.1:https://github.com/haidra-org/horde-sdk', 'prompt': 'A professional medical hospital logo for Rolland Memorial Hospital', 'params': {'sampler_name': <KNOWN_SAMPLERS.k_euler_a: 'k_euler_a'>, 'cfg_scale': 7.5, 'denoising_strength': 1, 'seed': None, 'height': 448, 'width': 448, 'seed_variation': None, 'post_processing': [], 'post_processing_order': <POST_PROCESSOR_ORDER_TYPE.facefixers_first: 'facefixers_first'>, 'karras': True, 'tiling': False, 'hires_fix': False, 'hires_fix_denoising_streng

✅ Image saved to images/hospital_logo_1739821440.png


NameError: name 'Cm' is not defined